# Neural networks auto-differentiation using PyTorch 2.0

In [ ]:
import os
import sys

sys.path.append(os.path.join(os.path.abspath(''), ".."))

import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.func import jacrev, jacfwd, hessian, vmap
from torch.nn import ELU

from nnbma.networks import FullyConnected

## Part 1: Differentiation of an analytical function

In this part, we will implement in PyTorch the following vectorial function

$ f: \left(\begin{array}{c} t_1\\ t_2 \end{array}\right) \longmapsto \left(\begin{array}{c} t_1+t_2\\ t_1^2+t_2^2\\ t_1^3 + t_2^3 \end{array}\right) $.

In [ ]:
n_inputs, n_outputs = 2, 3

def f(t: torch.Tensor) -> torch.Tensor:
    """
    Implements the (t1, t2) -> (t1+t1, t1^2+t2^2, t1^3+t2^3) function in PyTorch.
    This function takes batched inputs, so the inputs must have a shape [N, 2] where N is the batch size i.e. the number of inputs that are computed simultaneously.
    The output is of shape [N, 3].
    """
    if t.ndim != 2:
        raise ValueError(f"t must have 2 dimensions, not {t.ndim}")
    if t.shape[1] != n_inputs:
        raise ValueError(f"t.shape[1] must be {n_inputs}, not {t.shape[1]}")
    return torch.vstack((t.sum(1), (t**2).sum(1), (t**3).sum(1)))

For instance, we can plot a profile of this function:

In [ ]:
# TODO

We can also draw a bivariate plot:

In [ ]:
# TODO

## Part 2: Differentiation of a neural network

We will train a neural network to approximate the previous function. After that, we will compute the first and second derivatives of the network.

In [ ]:
# Training and test dataset

n_data = 1_000
test_frac = 0.20

a, b = -2, 2

In [ ]:
inputs = np.random.uniform(a, b, size=(n_data, n_inputs))
outputs = f(inputs)



In [ ]:
layers_sizes = [n_inputs, 1000, 1000, 1000, n_outputs]
activation = ELU()

model = FullyConnected(layers_sizes, activation)